<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [ ]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [32]:
import glob
import pandas as pd
import requests
from datetime import datetime
import io

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

In [12]:
url = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json",
       "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json",
       "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv")

html_data=[]
for n in url :
    r = requests.get(n)
    html_data.append(r.text) 

{"Name":{"35":"Banco Bilbao Vizcaya Argentaria","36":"Japan Post Bank","37":"The Bank of New York Mellon","38":"Shanghai Pudong Development Bank","39":"Industrial Bank (China)","40":"Bank of China (Hong Kong)","41":"Bank of Montreal","42":"Cr\u00e9dit Agricole","43":"DBS Bank","44":"Nordea","45":"Capital One","46":"NatWest Group","47":"Mizuho Financial Group","48":"Credit Suisse","49":"Postal Savings Bank of China","50":"China Minsheng Bank","51":"UniCredit","52":"China CITIC Bank","53":"Hang Seng Bank","54":"Soci\u00e9t\u00e9 G\u00e9n\u00e9rale","55":"Barclays","56":"Canadian Imperial Bank of Commerce","57":"Bank Central Asia","58":"Truist Financial","59":"Oversea-Chinese Banking Corp","60":"State Bank of India","61":"State Street","62":"Deutsche Bank","63":"KBC Bank","64":"Danske Bank","65":"Ping An Bank","66":"Standard Chartered","67":"United Overseas Bank","68":"QNB Group","69":"Bank Rakyat"},"Market Cap (US$ Billion)":{"35":"60.678","36":"60.525","37":"59.823","38":"58.796","39":"

In [59]:
rates = pd.read_csv(io.StringIO(html_data[2]))
rates.columns= ['Country', 'Rates']

# print(rates)

a = rates[rates['Country']=='AUD']

print(a)

  Country     Rates
0     AUD  1.297088


## Extract


### JSON Extract Function

This function will extract JSON files.


In [ ]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [79]:
columns=['Name','Market Cap (US$ Billion)']

In [94]:
def extract(source,columns):
    df = pd.read_json(io.StringIO(source))
    df.columns = columns
    return df

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [81]:
# Write your code here
exchange_rate = rates[rates['Country']=='GBP']
print(exchange_rate['Rates'])

9    0.732398
Name: Rates, dtype: float64


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [226]:
def transform(dataI,rate,columnN,ndecimals):
    data = dataI
    for index, row in data.iterrows():
        data.loc[index,'Market Cap (US$ Billion)'] = row['Market Cap (US$ Billion)']*rate.iloc[0,1]
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].round(decimals = ndecimals)
    name = 'Market Cap ({} Billion)'.format(rate.iloc[0,0])
    data.rename(columns = {'Market Cap (US$ Billion)':name}, inplace = True)
    return data
        
    

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [209]:
def load(df,name):
    result = df.to_csv(name,encoding='utf-8',index=False)
    return result

## Logging Function


Write the logging function <code>log</code> to log your data:


In [ ]:
def log(data):
    return data

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [227]:
market_data = extract(html_data[0],columns)
market_data1 = extract(html_data[1],columns)

market_data_g = pd.concat([market_data,market_data1],ignore_index=True)
print(market_data_g)

                                        Name  Market Cap (US$ Billion)
0                             JPMorgan Chase                   390.934
1    Industrial and Commercial Bank of China                   345.214
2                            Bank of America                   325.331
3                                Wells Fargo                   308.013
4                    China Construction Bank                   257.399
..                                       ...                       ...
100                             Ping An Bank                    37.993
101                       Standard Chartered                    37.319
102                     United Overseas Bank                    35.128
103                                QNB Group                    33.560
104                              Bank Rakyat                    33.081

[105 rows x 2 columns]


### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [204]:
market_data_g.head(5)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [228]:
market_data_gw = market_data_g



### Transform


Log the following  <code>"Transform phase Started"</code>


In [229]:
market_data_t = transform(market_data_gw,exchange_rate,3,3)
print(market_data_t)

                                        Name  Market Cap (GBP Billion)
0                             JPMorgan Chase                   286.319
1    Industrial and Commercial Bank of China                   252.834
2                            Bank of America                   238.272
3                                Wells Fargo                   225.588
4                    China Construction Bank                   188.519
..                                       ...                       ...
100                             Ping An Bank                    27.826
101                       Standard Chartered                    27.332
102                     United Overseas Bank                    25.728
103                                QNB Group                    24.579
104                              Bank Rakyat                    24.228

[105 rows x 2 columns]


<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [196]:
market_data_t.head(5)


,Name,Market Cap (GBP Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621


Log your data <code>"Transform phase Ended"</code>


In [197]:
market_data_t


,Name,Market Cap (GBP Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621
...,...,...
100,Ping An Bank,27.826013
101,Standard Chartered,27.332377
102,United Overseas Bank,25.727692
103,QNB Group,24.579291


### Load


Log the following `"Load phase Started"`.


Call the load function


In [ ]:
r = load(market_data_t,'market_data_t.csv')

Log the following `"Load phase Ended"`.


In [ ]:
# Write your code here


## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
